# Bikes & Cycling Accessories Organisation's Transactions Data Based Cohort Analysis

# Importing Libraries


# Libarary

In [ ]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

#import missingno as msno
#from textwrap import wrap

# Data loading

In [ ]:
transaction_df = pd.read_excel('/content/KPMG_VI_New_raw_data_update_final.xlsx','Transactions')
demographic_df = pd.read_excel('/content/KPMG_VI_New_raw_data_update_final.xlsx','CustomerDemographic')
address_df = pd.read_excel('/content/KPMG_VI_New_raw_data_update_final.xlsx','CustomerAddress')

## 1. Transactions


#### Step1. 讀取Dataset & 做基本處理和調整

In [ ]:
# Loading dataset
transaction_df = pd.read_excel('/content/KPMG_VI_New_raw_data_update_final.xlsx','Transactions')


In [ ]:
# View data
# 這裡的column name並不是真正的column name
# 可觀察到這裡讀出來的column name都是 unnamed..
transaction_df.head()

In [ ]:
# 用.iloc 根據整數位置進行索引和選擇數據
# 把column name的位置調整好
# 但又發現inex[0]這個位置也有column name, 那就要把這多餘的column name刪掉

transaction_df.columns = transaction_df.iloc[0]
transaction_df

In [ ]:
# 注意：刪除多餘的row!
# .drop(df.index[0], axis=0, inplace=True)
# 索引哪一行, 是沿著row進行, 並且確定更改
transaction_df.drop(index=transaction_df.index[0], axis=0, inplace=True)

In [ ]:
transaction_df

####Step2. 先觀察Dataset的基本資料

In [ ]:
# 總行數：20000
# Categorical Variable: online_order, order_status, brand, product_line, product_size,product_class
# Numerical Variable: list_price, standard_cost
# 其餘：transaction_id, product_id, customer_id, transaction_date, product_first_sold_date
transaction_df.describe()

In [ ]:
transaction_df.info()

#### Step3. 新增一列 Profit

*   因為有list_price 售價 和 standard_cost 成本



In [ ]:
# 新增一列：Profit
# 因為有成本和售價,才能多增加這一列去看分佈. 這樣又多了一個Numerical Variable

transaction_df['profit'] = transaction_df['list_price'] - transaction_df['standard_cost']
transaction_df

## 2. Demographic

#### Step1. 讀取Dataset & 做基本處理和調整

In [ ]:
# Loading dataset
demographic_df = pd.read_excel('/content/KPMG_VI_New_raw_data_update_final.xlsx','CustomerDemographic')

In [ ]:
demographic_df.head()

In [ ]:
demographic_df

####Step2. 新增一列age_group
*   因為age的數據太多, 所以設置成有區間的Categorical Variable

In [ ]:
# 1. 先把 age 轉成數字, errors='coerce'會把轉不了的值變成 NaN. 避免錯誤
demographic_df['age']= pd.to_numeric(demographic_df['age'], errors='coerce')

# 2. 設置要區隔的分組數
n_bins = 5

# 3. 等數區隔：用quct()自動將年齡依照人數分成N組
demographic_df['age_group'] = pd.qcut(demographic_df['age'], q=n_bins)

# 4. 查看結果
print(demographic_df['age_group'].value_counts().sort_index())

In [ ]:
# 用quct()是因為每組會等量又能自動計算分位數
# cut()不能自動調整分組, 但每組範圍是等寬平均

# 加上自定義的標籤
labels = ['20y-35y', '35y-45y','45y-50y','50y-60y','60y+']
demographic_df['age_group'] = pd.qcut(demographic_df['age'], q=n_bins, labels=labels)

print(demographic_df)

#### Step3. 查看數據

In [ ]:
demographic_df.describe()

In [ ]:
demographic_df.info()

## 3. Address

#### Step1. 讀取Dataset & 做基本處理和調整

In [ ]:
address_df = pd.read_excel('/content/KPMG_VI_New_raw_data_update_final.xlsx','CustomerAddress')

In [ ]:
address_df.head()

In [ ]:
address_df.columns = address_df.iloc[0]
address_df

In [ ]:
address_df.drop(index=address_df.index[0], axis=0, inplace=True)
address_df

#### Step2. 整理State這列的重複名稱
*   這列裡有重疊的正寫和縮寫的值, 一率合併成縮寫名稱

In [ ]:
address_df = address_df.replace("New South Wales","NSW")
address_df = address_df.replace("Victoria","VIC")

####Step3. 先觀察Dataset的基本資料

In [ ]:
address_df.describe()

In [ ]:
address_df.info()

# Data Cleaning : Checking & Organizing the Missing Value

## 1. Transaction

In [ ]:
print(transaction_df.info())

### (1) Numerical Variable

In [ ]:
print(transaction_df.isnull().values.sum())

In [ ]:
transaction_df = transaction_df.replace(" ",np.nan)

In [ ]:
transaction_df['standard_cost'] = transaction_df['standard_cost'].fillna(transaction_df['standard_cost'].median())

In [ ]:
print(transaction_df.isnull().values.sum())

In [ ]:
print(transaction_df.info())

###(2) Categorical Variable

In [ ]:
 transaction_df['product_line'].value_counts()

In [ ]:
transaction_df['product_line'].value_counts().index[0]

In [ ]:
# 對於每一列的categorical variable就可以寫for loop

print(transaction_df.info())
for col in transaction_df.columns:

    # Check if the column is of object type 代表是cate v
    # 判斷每一列是否為 object

    if transaction_df[col].dtypes == 'object':

        # Impute with the most frequent value 用fillna with 這列的眾數

        transaction_df[col] = transaction_df[col].fillna(transaction_df[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify 最後再計算看看還有無缺失值

print(transaction_df.isnull().values.sum())

In [ ]:
print(transaction_df.info())

## 2. Demographic

In [ ]:
print(demographic_df.info())

### (1) Numerical Variable

In [ ]:
print(demographic_df.isnull().values.sum())

In [ ]:
demographic_df = demographic_df.replace(" ",np.nan)

In [ ]:
demographic_df['tenure'] = demographic_df['tenure'].fillna(demographic_df['tenure'].median())
demographic_df['age'] = demographic_df['age'].fillna(demographic_df['age'].median())
print(demographic_df.isnull().values.sum())

### (2) Categorical Variable

In [ ]:
demographic_df['job_industry_category'].value_counts()

In [ ]:
demographic_df['job_industry_category'].value_counts().index[0]

In [ ]:
# 對於每一列的categorical variable就可以寫for loop

for col in demographic_df.columns:

    # Check if the column is of object type 代表是cate v
    # 判斷每一列是否為 object

    if demographic_df[col].dtypes == 'object':

        # Impute with the most frequent value 用fillna with 這列的眾數

        demographic_df[col] =  demographic_df[col].fillna(demographic_df[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify 最後再計算看看還有無缺失值

print(demographic_df.isnull().values.sum())

In [ ]:
print(demographic_df.info())

In [ ]:
demographic_df['age_group'].value_counts()

In [ ]:
demographic_df['age_group'].value_counts().index[0]

In [ ]:
if demographic_df['age_group'].dtypes == 'category':
    demographic_df['age_group'] = demographic_df['age_group'].fillna(demographic_df['age_group'].value_counts().index[0])
    print(demographic_df.isnull().values.sum())

In [ ]:
print(demographic_df.info())

# Join 3 Tables

In [ ]:
join_df = pd.merge(transaction_df, demographic_df, on='customer_id', how = 'inner').merge(address_df, on='customer_id', how = 'inner')
join_df

In [ ]:
join_df.info()

In [ ]:
join_df.describe()

# EDA - Transaction table merge customer table - dist - strip - rel - heatmap......

## 1. 單列、單一維度(前面都有寫過了,這裡省略)

##2. 兩個維度：x軸categorical variable, y軸numerical variable

*   Strip Plot
*   Box Plot
*   Count Plot



In [ ]:
# 兩個維度可以用 Strip Plot 和 Count Plot 來畫圖
# Strip Plot 如果看不出區別、不好畫圖的話, 可以換成Box Plot之類的

### (1) Strip Plot




In [ ]:
# 數據太分散，不適用

### (2) Box Plot

In [ ]:
# 1-1.
# x='brand', y='tenure'
sns.boxplot(x='brand', y='tenure', data=join_df)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1-2. x='brand', y='past_3_years_bike_related_purchases'
sns.boxplot(x='brand', y='past_3_years_bike_related_purchases', data=join_df)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 2-1. x='product_line', y='tenure'
sns.boxplot(x='product_line', y='tenure', data=join_df)

In [ ]:
# 2-2. x='product_line', y='past_3_years_bike_related_purchases'
sns.boxplot(x='product_line', y='past_3_years_bike_related_purchases', data=join_df)

In [ ]:
# 3-1. x='job_insudstry_category', y='profit'
plt.figure(figsize=(12,6))
sns.boxplot(x='job_industry_category', y='profit', data=join_df)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3-2. x='age_group', y='profit'
plt.figure(figsize=(12,6))
sns.boxplot(x='age_group', y='profit', data=join_df)


In [ ]:
# 4-1. x= 'state', y='profit'
sns.boxplot(x='state', y='profit', data=join_df)

In [ ]:
# 4-2. x='property_valuation', y='profit'
plt.figure(figsize=(12,6))
sns.boxplot(x='property_valuation', y='profit', data=join_df)

In [ ]:
# 5-1. x='state', y='tenure'
sns.boxplot(x='state', y='tenure', data=join_df)

In [ ]:
# 5-2. x='property_valuation', y='tenure'
plt.figure(figsize=(12,6))
sns.boxplot(x='property_valuation', y='tenure', data=join_df)

In [ ]:
# 6-1. x='state', y='past_3_years_bike_related_purchases'
sns.boxplot(x='state', y='past_3_years_bike_related_purchases', data=join_df)

In [ ]:
# 6-2. x='property_valuation', y='past_3_years_bike_related_purchases'
plt.figure(figsize=(12,6))
sns.boxplot(x='property_valuation', y='past_3_years_bike_related_purchases', data=join_df)

### (3) Count Plot ( x軸 ＆ hue 都是Categorical )
* 因為y軸是計算總數, 自然屬於numerical

In [ ]:
# 1-1. x='brand', hue='gender'
plt.figure(figsize=(10,6))
sns.countplot(x='brand', data=join_df, hue='gender')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1-2. x='brand', hue='wealth_segment'
plt.figure(figsize=(10,6))
sns.countplot(x='brand', data=join_df, hue='wealth_segment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1-3. x='brand', hue='state'
plt.figure(figsize=(10,6))
sns.countplot(x='brand', data=join_df, hue='state')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 2-1. x='product_line', hue='gender'
sns.countplot(x='product_line', data=join_df, hue='gender')


In [ ]:
# 2-2. x='product_line', hue='wealth_segment'
sns.countplot(x='product_line', data=join_df, hue='wealth_segment')

In [ ]:
# 2-3. x='product_line', hue='state'
sns.countplot(x='product_line', data=join_df, hue='state')


In [ ]:
# 3-1. x='job_industry_category', hue='product_class'
plt.figure(figsize=(10,6))
sns.countplot(x='job_industry_category', data=join_df, hue='product_class')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3-2. x='job_industry_category', y ='product_size' 和上圖沒有太大的差異

In [ ]:
# 3-3. x='job_industry_category', y ='state'
plt.figure(figsize=(10,6))
sns.countplot(x='job_industry_category', data=join_df, hue='state')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 4-1. x='age_group', hue='product_class'

sns.countplot(x='age_group', data=join_df, hue='product_class')

In [ ]:
# 4-2. x='age_group', hue='state'
sns.countplot(x='age_group', data=join_df, hue='state')

In [ ]:
# 5-1. x='property_valuation', hue='product_class'
plt.figure(figsize=(20,6))
sns.countplot(x='property_valuation', data=join_df, hue='product_class')

In [ ]:
# 5-2. x='property_valuation', hue='gender'
plt.figure(figsize=(20,6))
sns.countplot(x='property_valuation', data=join_df, hue='gender')

In [ ]:
# 5-3. x='property_valuation', hue='wealth_segment'
plt.figure(figsize=(20,6))
sns.countplot(x='property_valuation', data=join_df, hue='wealth_segment')

## 3. 三個維度: x軸categorical variable, y軸numerical variable, hue 顏色區分

*   Strip Plot 散點圖
*   Box Plot 箱型圖(聚合型)
*   Bar Plot 條形圖




### (1) Strip Plot: 太分散了, 意義不大. 大部分都是長橫條圖

In [ ]:
# 顏色區分的顆粒度也不明顯

###(2) Box Plot

In [ ]:
# 1-1. x='brand', y='profit', hue='gender'
plt.figure(figsize=(12,6))
sns.boxplot(x='brand', y='profit', data=join_df, hue='gender')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1-2. x='brand', y='profit', hue='wealth_segment'
plt.figure(figsize=(12,6))
sns.boxplot(x='brand', y='profit', data=join_df, hue='wealth_segment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1-3. x='brand', y='profit', hue='state'
plt.figure(figsize=(12,6))
sns.boxplot(x='brand', y='profit', data=join_df, hue='state')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 2-1. x='product_line', y='profit', hue='gender'
sns.boxplot(x='product_line', y='profit', data=join_df, hue='gender')

In [ ]:
# 2-2. x='product_line', y='profit', hue='wealth_segment'
sns.boxplot(x='product_line', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 2-3. x='product_line', y='profit', hue='state'
sns.boxplot(x='product_line', y='profit', data=join_df, hue='state')

In [ ]:
# 3-1. x='product_class', y='profit', hue='gender'
sns.boxplot(x='product_class', y='profit', data=join_df, hue='gender')

In [ ]:
# 3-2. x='product_class', y='profit', hue='wealth_segment'
sns.boxplot(x='product_class', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 3-3. x='product_class', y='profit', hue='state'
sns.boxplot(x='product_class', y='profit', data=join_df, hue='state')

In [ ]:
# 4-1. x='job_industry_category', y='profit', hue='product_class'
plt.figure(figsize=(12,6))
sns.boxplot(x='job_industry_category', y='profit', data=join_df, hue='product_class')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 4-2. x='job_industry_category', y='profit', hue='gender'
plt.figure(figsize=(12,6))
sns.boxplot(x='job_industry_category', y='profit', data=join_df, hue='gender')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 4-3. x='job_industry_category', y='profit', hue='wealth_segment'
plt.figure(figsize=(12,6))
sns.boxplot(x='job_industry_category', y='profit', data=join_df, hue='wealth_segment')
plt.xticks(rotation=45)
plt.tight_layout()

In [ ]:
# 4-4. x='job_industry_category', y='profit', hue='state'
plt.figure(figsize=(12,6))
sns.boxplot(x='job_industry_category', y='profit', data=join_df, hue='state')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 5-1. x='age_group', y='profit', hue='product_class'
plt.figure(figsize=(12,6))
sns.boxplot(x='age_group', y='profit', data=join_df, hue='product_class')

In [ ]:
# 5-2. x='age_group', y='profit', hue='gender'
plt.figure(figsize=(12,6))
sns.boxplot(x='age_group', y='profit', data=join_df, hue='gender')

In [ ]:
# 5-3. x='age_group', y='profit', hue='wealth_segment'
plt.figure(figsize=(12,6))
sns.boxplot(x='age_group', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 5-4. x='age_group', y='profit', hue='state'
plt.figure(figsize=(12,6))
sns.boxplot(x='age_group', y='profit', data=join_df, hue='state')

In [ ]:
# 6-1. x='property_valuation', y='profit', hue='product_class'
plt.figure(figsize=(20,6))
sns.boxplot(x='property_valuation', y='profit', data=join_df, hue='product_class')

In [ ]:
# 6-2. x='property_valuation', y='profit', hue='gender'
plt.figure(figsize=(20,6))
sns.boxplot(x='property_valuation', y='profit', data=join_df, hue='gender')

In [ ]:
# 6-3. x='property_valuation', y='profit', hue='wealth_segment'
plt.figure(figsize=(20,6))
sns.boxplot(x='property_valuation', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 6-4. x='property_valuation', y='profit', hue='state'
plt.figure(figsize=(20,6))
sns.boxplot(x='property_valuation', y='profit', data=join_df, hue='state')

### (3) Bar Plot：沒有太多黑線的話, 看高度

In [ ]:
# 1-1. x='brand', y='profit', hue='gender'
plt.figure(figsize=(12,6))
sns.barplot(x='brand', y='profit', data=join_df, hue='gender')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1-2. x='brand', y='profit', hue='wealth_segment'
plt.figure(figsize=(12,6))
sns.barplot(x='brand', y='profit', data=join_df, hue='wealth_segment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 1-3. x='brand', y='profit', hue='state'
plt.figure(figsize=(12,6))
sns.barplot(x='brand', y='profit', data=join_df, hue='state')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 2-1. x='product_line', y='profit', hue='gender’
sns.barplot(x='product_line', y='profit', data=join_df, hue='gender')

In [ ]:
# 2-2. x='product_line', y='profit', hue='wealth_segment’
sns.barplot(x='product_line', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 2-3. x='product_line', y='profit', hue='state’
sns.barplot(x='product_line', y='profit', data=join_df, hue='state')

In [ ]:
# 3-1. x='product_class', y='profit', hue='gender’
sns.barplot(x='product_class', y='profit', data=join_df, hue='gender')

In [ ]:
# 3-2. x='product_class', y='profit', hue='wealth_segment’
sns.barplot(x='product_class', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 3-3. x='product_class', y='profit', hue='state’
sns.barplot(x='product_class', y='profit', data=join_df, hue='state')

In [ ]:
# 4-1. x='job_industry_category', y='profit', hue='product_class’
plt.figure(figsize=(12,6))
sns.barplot(x='job_industry_category', y='profit', data=join_df, hue='product_class')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 4-2. x='job_industry_category', y='profit', hue='gender’
plt.figure(figsize=(12,6))
sns.barplot(x='job_industry_category', y='profit', data=join_df, hue='gender')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 4-3. x='job_industry_category', y='profit', hue='wealth_segment’
plt.figure(figsize=(12,6))
sns.barplot(x='job_industry_category', y='profit', data=join_df, hue='wealth_segment')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 4-4. x='job_industry_category', y='profit', hue='state’
plt.figure(figsize=(12,6))
sns.barplot(x='job_industry_category', y='profit', data=join_df, hue='state')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 5-1. x='age_group', y='profit', hue='product_class’
plt.figure(figsize=(12,6))
sns.barplot(x='age_group', y='profit', data=join_df, hue='product_class')

In [ ]:
# 5-2. x='age_group', y='profit', hue='gender’
plt.figure(figsize=(12,6))
sns.barplot(x='age_group', y='profit', data=join_df, hue='gender')

In [ ]:
# 5-3. x='age_group', y='profit', hue='wealth_segment’
plt.figure(figsize=(12,6))
sns.barplot(x='age_group', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 5-4. x='age_group', y='profit', hue='state’
plt.figure(figsize=(12,6))
sns.barplot(x='age_group', y='profit', data=join_df, hue='state')

In [ ]:
# 6-1. x='property_valuation', y='profit', hue='product_class’
plt.figure(figsize=(20,6))
sns.barplot(x='property_valuation', y='profit', data=join_df, hue='product_class')

In [ ]:
# 6-2. x='property_valuation', y='profit', hue='gender’
plt.figure(figsize=(20,6))
sns.barplot(x='property_valuation', y='profit', data=join_df, hue='gender')

In [ ]:
# 6-3. x='property_valuation', y='profit', hue='wealth_segment’
plt.figure(figsize=(20,6))
sns.barplot(x='property_valuation', y='profit', data=join_df, hue='wealth_segment')

In [ ]:
# 6-4. x='property_valuation', y='profit', hue='state’
plt.figure(figsize=(20,6))
sns.barplot(x='property_valuation', y='profit', data=join_df, hue='state')

## 4. 聚合分佈 (三個維度)：x軸 Categorical variable, y軸 Numerical variable


*   Box Plot
*   Violin Plot
* 如果做了Box Plot就不用做Violin Plot



### Violin Plot

In [ ]:
# 前面畫過Box Plot了

## 5. Numerical 和 Numerical variable的關係

### (1) Joint Plot

In [ ]:
# 1-1. x='profit', y='tenure'
# 1-2. x-'profit', y='past_3_years_bike_related_purchases'
# 2-1. x='list_price', y='tenure'
# 3-1. x='standard_cost', y='tenure'
# 這兩個做出來的圖都太分散, 沒有意義

### (2) Point Plot (x軸以兩個為一組比對最適用這種畫法)

*   x軸和hue都放Categorical Variable
*   因為point plot是看兩組人對於同一個變量sensitivity的程度, 如果是Num會有比較多高低的曲線去代表分佈.
*   但是point plot是靠著傾斜程度去觀察他的sensitivity, 所以x軸和hue才放categorical.只有y軸放numerical




In [ ]:
print(join_df.columns)

In [ ]:
# 1-1. x='online_order', y='tenure', hue='order_status'
sns.pointplot(x='online_order', y='tenure', data=join_df, hue='order_status')

In [ ]:
# 1-2. x='online_order', y='past_3_years_bike_related_purchases', hue='order_status'
sns.pointplot(x='online_order', y='past_3_years_bike_related_purchases', data=join_df, hue='order_status')

In [ ]:
#1-3. x='online_order', y='tenure', hue='owns_car'
sns.pointplot(x='online_order', y='tenure', data=join_df, hue='owns_car')

In [ ]:
# 1-4. x='online_order', y='past_3_years_bike_related_purchases', hue='owns_car'
sns.pointplot(x='online_order', y='past_3_years_bike_related_purchases', data=join_df, hue='owns_car')

In [ ]:
# 2-1. x='order_status',  y='tenure', hue='owns_car'
# 2-2. x='order_status',  y='past_3_years_bike_related_purchases', hue='online_order'
# 沒有明顯差異

In [ ]:
# Q: 像3-1這種結果是不是比較能看得出差異？

In [ ]:
# 3-1. x='owns_car', y ='profit', hue='online_order'
sns.pointplot(x='owns_car', y='profit', data=join_df, hue='online_order')

In [ ]:
# 3-2. x='owns_car', y ='profit', hue='order_status'
sns.pointplot(x='owns_car', y='profit', data=join_df, hue='order_status')

In [ ]:
# 3-3. x='owns_car', y ='standard_cost', hue='online_order'
sns.pointplot(x='owns_car', y='standard_cost', data=join_df, hue='online_order')

In [ ]:
# 3-3. x='owns_car', y ='list_price', hue='online_order'
sns.pointplot(x='owns_car', y='list_price', data=join_df, hue='online_order')

In [ ]:
# 3-4. x='owns_car', y ='tenure', hue='online_order'
sns.pointplot(x='owns_car', y='tenure', data=join_df, hue='online_order')

In [ ]:
# 3-5. x='owns_car', y ='past_3_years_bike_related_purchases', hue='online_order'
sns.pointplot(x='owns_car', y='past_3_years_bike_related_purchases', data=join_df, hue='online_order')

## 6. 多個維度: 多個 Num v 和 Cate v之間的關係 (先畫Cat Plot就好)

### 如果 Relational Plot太分散了,就用Cat Plot 聚合圖形表示

*   kind = "boxen"
*   kind = "bar"



### Cat Plot：kind能改成聚合型Box Plot的圖

In [ ]:
# 1-1. x='brand', y='profit'
# hue = 'gender', col='wealth_segment', row='state'
sns.set(style='whitegrid', palette='pastel')


plot_1 = sns.catplot(x='brand', y='profit', data=join_df, hue='gender', col='wealth_segment', row='state',kind='bar')
plot_1.set(xlabel="brand",ylabel="profit")

for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()



In [ ]:
# 1-2. x='brand', y='profit'
# hue = 'gender', col='wealth_segment', row='product_class'
sns.set(style='whitegrid', palette='pastel')

plot_1 = sns.catplot(x='brand', y='profit', data=join_df, hue='gender', col='wealth_segment', row='product_class',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
# 1-3. x='brand', y='profit'
# hue = 'online_order', col='wealth_segment', row='state'
sns.set(style='whitegrid', palette='pastel')

plot_1 = sns.catplot(x='brand', y='profit', data=join_df, hue='online_order', col='wealth_segment', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
# 1-4. x='brand', y='profit'
# hue = 'owns_car', col='wealth_segment', row='state'
sns.set(style='whitegrid', palette='pastel')

plot_1 = sns.catplot(x='brand', y='profit', data=join_df, hue='owns_car', col='wealth_segment', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
# 2-1. x='product_line', y='profit'
# hue = 'gender', col='wealth_segment', row='state'
sns.set(style='whitegrid', palette='pastel')

plot_1 = sns.catplot(x='product_line', y='profit', data=join_df, hue='gender', col='wealth_segment', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
# 2-2. x='product_line', y='profit'
# hue = 'online_order', col='wealth_segment', row='state'
sns.set(style='whitegrid', palette='pastel')
plot_1 = sns.catplot(x='product_line', y='profit', data=join_df, hue='online_order', col='wealth_segment', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
# 2-3. x='product_line', y='profit'
# hue = 'online_order', col='product_class', row='state'
sns.set(style='whitegrid', palette='pastel')
plot_1 = sns.catplot(x='product_line', y='profit', data=join_df, hue='online_order', col='product_class', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
# 3-1. x='age_group', y='profit'
# hue = 'online_order', col='product_class', row='state'
sns.set(style='whitegrid', palette='pastel')
plot_1 = sns.catplot(x='age_group', y='profit', data=join_df, hue='online_order', col='product_class', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

In [ ]:
# 3-2. x='age_group', y='profit'
# hue = 'gender', col='product_class', row='state'
sns.set(style='whitegrid', palette='pastel')
plot_1 = sns.catplot(x='age_group', y='profit', data=join_df, hue='gender', col='product_class', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()


In [ ]:
# 3-3. x='age_group', y='profit'
# hue = 'wealth_segment', col='product_class', row='state'
sns.set(style='whitegrid', palette='pastel')
plot_1 = sns.catplot(x='age_group', y='profit', data=join_df, hue='wealth_segment', col='product_class', row='state',kind='bar')
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()


## 7. 多個Numerical v之間的相關性

### Heat Plot

In [ ]:
# 1.
join_df[["tenure","standard_cost","profit"]]

In [ ]:
join_df[["tenure","standard_cost","profit"]].corr()

In [ ]:
sns.heatmap(join_df[["tenure","standard_cost","profit"]].corr(), annot=True)

In [ ]:
#2.
join_df[["past_3_years_bike_related_purchases","standard_cost","profit"]]

In [ ]:
join_df[["past_3_years_bike_related_purchases","standard_cost","profit"]].corr()

In [ ]:
sns.heatmap(join_df[["past_3_years_bike_related_purchases","standard_cost","profit"]].corr(), annot=True)

In [ ]:
# 3.
join_df[["list_price","profit","tenure"]]

In [ ]:
join_df[["list_price","profit","tenure"]].corr()

In [ ]:
sns.heatmap(join_df[["list_price","profit","tenure"]].corr(), annot=True)

In [ ]:
#4.
join_df[["past_3_years_bike_related_purchases","profit","tenure"]]

In [ ]:
join_df[["past_3_years_bike_related_purchases","profit","tenure"]].corr()

In [ ]:
sns.heatmap(join_df[["past_3_years_bike_related_purchases","profit","tenure"]].corr(), annot=True)

# Cohort Analysis

## Assigned the Cohorts & Calculated the Monthly Offset

In [ ]:
# Transaction_df 有 transaction_date, 也就是說有一整年的data, 所以 Cohort by Month
# 總共有幾千個用戶, 又有20000筆transaction = 一人一年有多筆transaction

# 所以看這個人在這一年裡發生第一筆transaction的月份,就是這個人的Cohort Month

In [ ]:
# 將交易日期轉成datetime的格式
# 並從其中某一筆交易取出「該筆交易所屬月份的第一天」
transaction_df['transaction_date'] = pd.to_datetime(transaction_df['transaction_date'])
dt.datetime(transaction_df['transaction_date'][1].year, transaction_df['transaction_date'][1].month, 1)

### Step1. 先求 Transaction Month

In [ ]:
# 先要把transaction_date變成transaction_month. 可利用datetime()
# datetime(year, month ,1)把統一的日期都變為1號
# A function that will parse the date Time based cohort:  1 day of month

def get_month(x):
  return dt.datetime(x.year, x.month, 1)

# Create transaction_date column based on month and store in TransactionMonth
# 用apply()剛剛自定義的函數,把transaction_date變成transaction_month, 就是把每筆transaction都變成當月的1號
# 現在有了transaction_month就可以求cohort month了

transaction_df['TransactionMonth'] = transaction_df['transaction_date'].apply(get_month)
transaction_df

### HW: .min()和.transform('min)的區別？
* .min(): 是按照customer_id當index排列出來, 但是少於總行數2萬筆.
* .transform('min'): 是把每一筆transaciton都當作一個index, 排列出所有2萬筆的transcation. 只不過會有duplicate data.後面用unique()就可以調整

### Step 2. 再求 Cohort Month

In [ ]:
# 每一個customer_id,在所有歷史紀錄裡發生的第一筆transaction所在的月份
# 就是groupby customer_id 對於 transaction month 求 min

transaction_df.groupby('customer_id')['TransactionMonth'].min()

In [ ]:
transaction_df.groupby('customer_id')['TransactionMonth'].transform('min')

In [ ]:
# Grouping by customer_id and select the InvoiceMonth value
# Assigning a minimum InvoiceMonth value to the dataset

# 知道每一行這個customer_id 所有的transcation以後,這個id的第一筆transaction所在的月份
# 增加一列叫做cohort month


transaction_df['CohortMonth'] = transaction_df.groupby('customer_id')['TransactionMonth'].transform('min')

# printing top 5 rows

print(transaction_df.head())

In [ ]:
# 舉例講解：
# customer_id是2950的這個人,他所在的transaction moth是02-01,他的cohort month也是02-01
# 證明2950的這個人的第一筆transaction就是發生在2月份

# customer_id是3120的這個人,他所在的transaction month是05-01,他的cohort month是01-01
# 證明3120的這個人的第一筆transaction是發生在1月份

### Step 3. 求 Cohort Index

In [ ]:
# 現在有每一個customer_id也就是每個人的cohort month, 已經排列出每個人的第一筆交易紀錄的月份
# 接著就要求cohort index, 就是有這一整年的交易紀錄,所以用每個month來看差異
# 觀察每個月購買數量的對比

In [ ]:
# Q: 這裡的code看不懂
# A: 給任何一個column, 自動偵測年月日,
#    放回到transaction_df裡.一個歸類在transactionMonth,一個歸類在CohortMonth因為日不怎麼用得到,所以用下底線替換掉


def get_date_int(df, column):
    year = df[column].dt.year
    month = df[column].dt.month
    day = df[column].dt.day
    return year, month, day

# Getting the integers for date parts from the `InvoiceDay` column

transcation_year, transaction_month, _ = get_date_int(transaction_df, 'TransactionMonth')

# Getting the integers for date parts from the `CohortDay` column

cohort_year, cohort_month, _ = get_date_int(transaction_df, 'CohortMonth')

In [ ]:
# 那cohort index要怎麼求？
# 用transaction month的這一列, 證明這個人在這個月還在active
# 距離cohort month發生第一筆交易紀錄的月份是“第幾個月”,即是他的cohort index
# 證明這個人在第幾個月還在active

In [ ]:
#  Get the  difference in years 減掉年份的差異＊12

years_diff = transcation_year - cohort_year

# Calculate difference in months 減掉月份的差異 + 1
# 如果想把還在active的當月作為cohort month 1的話,就 +1
# 如果想從 cohort month 0 開始算的話,就不用加了

months_diff = transaction_month - cohort_month
""" Extract the difference in months from all previous values
 "+1" in addeded at the end so that first month is marked as 1 instead of 0 for easier interpretation.
 """
transaction_df['CohortIndex'] = years_diff * 12 + months_diff  + 1
transaction_df.head(5)

### Step 4. 計算總人口數 Number of Customer id

In [ ]:
# 有了transaction month, cohort month之後, 現在只差熱力圖格子裡的留存率.就要算留存率就先算出 “總人數的變化”.
# 那要怎麼算在cohort的 month1、month2… 的 “總人數”呢？

In [ ]:
# Counting daily active user from each chort
# Counting number of unique customer Id's falling in each group of CohortMonth and CohortIndex

# 就是 group by cohort month 和 cohort index 再count unique customer_id.

cohort_data = transaction_df.groupby(['CohortMonth', 'CohortIndex']).nunique()['customer_id'].reset_index()

# Printing top 5 rows of Dataframe

cohort_data.head(5)

In [ ]:
# 講解：
# 跑出來的結果就是3列N行
# 同一批人
# Cohort month 的第1個 月份, 一共有1358個unique customer id
# 在這1358個人裡面, Cohort month 的第2個 月份, 一共有497個人還在購買自行車(1358個人裡的497個人還在active)

#注意一下第5個月份, 繼續購買的人數增加了, 表示在1358個人裡, 有些人在第3、4個月都沒有購買. 但是在第5個月購買了

## Calculate business metrics: Retention rate

### Step 5. 整理成Pivot Table & 計算留存率

In [ ]:
# 這裡用 Pivot Table 做整理
# 可以一目瞭然. 一次看清楚所有data.

In [ ]:
# index放 Cohort Month, 就是第一筆transaction購買紀錄的月份
# columns放 Cohort Inedex, 第1個月份到第12個月份, 按照月份來看
# value 放的是 Customer id, 標出每個月份還持續購買的總人數


cohort_data.pivot(index='CohortMonth',  columns ='CohortIndex', values = 'customer_id')

In [ ]:
cohort_counts = cohort_data.pivot(index='CohortMonth',  columns ='CohortIndex', values = 'customer_id')

# 計算留存率！ 每個值去除以他們一橫行的第一列的總人口數
# 先把第1列提取出來

cohort_sizes = cohort_counts.iloc[:,0]

# 把整個table, 一橫行都除以他們的第一列
retention = cohort_counts.divide(cohort_sizes, axis=0)

# Coverting the retention rate into percentage and Rounding off.
retention.round(3)*100

## Visualizing the retention rate

In [ ]:
retention = retention.round(3)*100
retention.index = retention.index.strftime('%Y-%m')
# Initialize the figure
plt.figure(figsize=(16, 10))
# Adding a title
plt.title('Average Standard Cost: Monthly Cohorts', fontsize = 14)
# Creating the heatmap
sns.heatmap(retention, annot = True,vmin = 0.0, vmax =100,cmap="YlGnBu", fmt='g')
plt.ylabel('Cohort Month')
plt.xlabel('Cohort Index')
plt.yticks( rotation=360)
plt.show()

In [ ]:
# 這裡Cohort Analysis就做完了
# 看到留存率的變化現象, 就要找出變化的原因！利用Segmentation Analysis
# 挑變化明顯、差距大的, 找出上升和下降的原因
# Sementation Analysis去filter 上升和下降的原因.

In [ ]:
join_df = pd.merge(transaction_df, demographic_df, on='customer_id', how = 'inner').merge(address_df, on='customer_id', how = 'inner')
join_df.head()

# Segmentation Anaylsis

*   Root Cause Analysis of the Retention Rate
*   Analysing the changing reasons



## Case 1: 2024-07 52.5% -> 28.8%

### 提取 Data

In [ ]:
# 把join_df裡的customer_id 和 profit用group by 分組 並且 加總每個customer_id總共profit有多少. 最後重新整理index
# df1裡新增一列叫做 rank_M, 按照profit做排名 並 倒敘

df1 = join_df.groupby(['customer_id'])['profit'].sum().reset_index()
df1['rank_M'] = join_df['profit'].rank(ascending = False)

In [ ]:
df_52 = join_df[(join_df['CohortMonth'] == '2024-07-01') & (join_df['CohortIndex'] == 2)]

In [ ]:
df_28 = join_df[(join_df['CohortMonth'] == '2024-07-01') & (join_df['CohortIndex'] == 3)]

In [ ]:
df_52=df_52.copy()
df_28=df_28.copy()

### 單一維度：y = 'profit'

In [ ]:
# Q: 單一維度的displot, countplot都用不了

In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6), sharey=True)

sns.boxplot(data=df_52, y='profit', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - Augest")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.boxplot(data=df_28, y='profit', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - September")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

### 兩個維度：x軸cate, y軸num

*   Strip Plot
*   Box Plot
*   Count Plot (x & hue都是cate)



In [ ]:
# Box Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 6), sharey=True)


sns.boxplot(data=df_52, x='brand', y='profit', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - Augest")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")


sns.boxplot(data=df_28, x='brand', y='profit', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - September")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

In [ ]:
# Count Plot

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)


sns.countplot(data=df_52, x='brand', hue='gender', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")


sns.countplot(data=df_28, x='brand', hue='gender', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

### 三個維度： ：x軸cate, y軸num, hue
*   Strip Plot 散點圖
*   Box Plot 箱型圖(聚合型)
*   Bar Plot 條形圖

In [ ]:
# Strip Plot

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)


sns.stripplot(data=df_52, x='brand', y='profit', hue='gender', ax=axes[0], color='skyblue', size = 10 )
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")


sns.stripplot(data=df_28, x='brand', y='profit', hue='gender', ax=axes[1], color='lightcoral', size = 10 )
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

In [ ]:
# Bar Plot

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)


sns.barplot(data=df_52, x='brand', y='profit', hue='gender', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")


sns.barplot(data=df_28, x='brand', y='profit', hue='gender', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

### 三個維度：x軸 ＆ y軸 num, hue

*   Joint Plot
*   Point Plot



In [ ]:
print(join_df.columns)

In [ ]:
# Joint Plot
# Q: 要怎麼變成一橫行？
# A: Joint plot是Seaborn中自帶圖表佈局的函數. 不接受ax參數u,也無法直接嵌入subplot之中

sns.jointplot(data=df_52, x='standard_cost', y='profit', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")


sns.jointplot(data=df_28, x='standard_cost', y='profit', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")


plt.tight_layout()
plt.show()

In [ ]:
# 相較之下用Scatter能呈現成本和獲利的關係

fig, axes = plt.subplots(1, 2, figsize=(12, 5))  # 一橫行兩圖

# 左邊圖
sns.scatterplot(data=df_52, x='standard_cost', y='profit', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")

# 右邊圖
sns.scatterplot(data=df_28, x='standard_cost', y='profit', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")  # 不顯示 y label
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

### 多個維度： Cat Plot

In [ ]:

sns.set(style='whitegrid', palette='pastel')

plot_1=sns.catplot(x='brand', y='profit', data=df_52, hue='gender', col='wealth_segment', row='state',kind='bar')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")
for ax in plot_1.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()



sns.set(style='whitegrid', palette='pastel')

plot_2=sns.catplot(x='brand', y='profit', data=df_28, hue='gender', col='wealth_segment', row='state',kind='bar')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")
for ax in plot_2.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

### 多個Num V: Heat Plot

In [ ]:
df_52[["tenure","standard_cost","profit"]].corr()
df_28[["tenure","standard_cost","profit"]].corr()

In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), sharey=True)


sns.heatmap(df_52[["tenure","standard_cost","profit"]].corr(), annot=True, ax=axes[0])
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_xlabel("2024-07 Cohort Group")


sns.heatmap(df_28[["tenure","standard_cost","profit"]].corr(), annot=True, ax=axes[1])
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

## Case 2: 2024-10 15% -> 50%

### 提取 Data

In [ ]:
df1 = join_df.groupby(['customer_id'])['profit'].sum().reset_index()
df1['rank_M'] = join_df['profit'].rank(ascending = False)

In [ ]:
df_15 = join_df[(join_df['CohortMonth'] == '2024-10-01') & (join_df['CohortIndex'] == 2)]
df_50 = join_df[(join_df['CohortMonth'] == '2024-10-01') & (join_df['CohortIndex'] == 3)]

### 單一維度：y = 'profit'

In [ ]:
# Box Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6), sharey=True)

sns.boxplot(data=df_15, y='profit', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (15%) - October")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-10 Cohort Group")

sns.boxplot(data=df_50, y='profit', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (50%) - November")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-10 Cohort Group")

plt.tight_layout()
plt.show()

### 兩個維度：x軸cate, y軸num

*   Strip Plot
*   Box Plot
*   Count Plot (x & hue都是cate)

In [ ]:
# Box Plot

sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.boxplot(data=df_15, x='brand', y='profit', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (15%) - October")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-10 Cohort Group")

sns.boxplot(data=df_50, x='brand', y='profit', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (50%) - November")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-10 Cohort Group")

plt.tight_layout()
plt.show()

In [ ]:
# Count Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.countplot(data=df_15, x='brand', hue='gender', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (15%) - October")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-10 Cohort Group")

sns.countplot(data=df_50, x='brand', hue='gender', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (50%) - November")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-10 Cohort Group")

plt.tight_layout()
plt.show()

### 三個維度： ：x軸cate, y軸num, hue
*   Strip Plot 散點圖
*   Box Plot 箱型圖(聚合型)
*   Bar Plot 條形圖

In [ ]:
# Bar Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.barplot(data=df_15, x='brand', y='profit', hue='gender', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (15%) - October")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-10 Cohort Group")

sns.barplot(data=df_50, x='brand', y='profit', hue='gender', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (50%) - November")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-10 Cohort Group")

plt.tight_layout()
plt.show()

### 三個維度：x軸 ＆ y軸 num, hue

*   Joint Plot
*   Point Plot

In [ ]:
# 沒有意義

### 多個維度： Cat Plot

In [ ]:
# 沒有特別顯示有效的差異

### 多個Num V: Heat Plot

In [ ]:
df_15[["tenure","standard_cost","profit"]].corr()
df_50[["tenure","standard_cost","profit"]].corr()

In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), sharey=True)

sns.heatmap(df_15[["tenure","standard_cost","profit"]].corr(), annot=True, ax=axes[0])
axes[0].set_title("Cohort Index 2 (15%) - October")
axes[0].set_xlabel("2024-10 Cohort Group")

sns.heatmap(df_50[["tenure","standard_cost","profit"]].corr(), annot=True, ax=axes[1])
axes[1].set_title("Cohort Index 3 (50%) - November")
axes[1].set_xlabel("2024-10 Cohort Group")

plt.tight_layout()
plt.show()

# RFM Model 顧客價值分析模型

*   Recency : 最近一次購買距今多久？
*   Frequency : 一段時間內的購買次數
*   Monetary : 購買總金額



## Recency Rank

In [ ]:
# 取得每個客戶最近一次的交易日期
recency_df = transaction_df.groupby('customer_id')['transaction_date'].max().reset_index()

In [ ]:
# 依照最近期的時間由最近到最久遠排序
recency_df = recency_df.sort_values(by='transaction_date', ascending=False).reset_index(drop=True)

In [ ]:
# 新增一列叫 frecency_rank 加入排名的column
recency_df['recency_rank'] = recency_df['transaction_date'].rank(method='first', ascending=False)
recency_df.head()

## Frequncy Rank

In [ ]:
# 計算每一位客戶在今年有多少筆交易
frequency_df = transaction_df.groupby('customer_id')['transaction_id'].nunique().reset_index()
frequency_df.columns = ['customer_id', 'frequency']
frequency_df

In [ ]:
# 根據頻率次數由高到低排序
frequency_df = frequency_df.sort_values(by='frequency', ascending=False).reset_index(drop=True)
frequency_df

In [ ]:
# 新增一列叫 frequency_rank 加入排名的column
frequency_df['frequency_rank'] = frequency_df['frequency'].rank(method='first', ascending=False)
frequency_df

## Monetary Rank

In [ ]:
# 計算每位客戶消費而賺取的盈利總數
monetary_df = transaction_df.groupby('customer_id')['profit'].sum().reset_index()
monetary_df

In [ ]:
# 依照盈利的金格由最高到最低排序
monetary_df = monetary_df.sort_values(by='profit', ascending=False).reset_index(drop=True)
monetary_df

In [ ]:
# 新增一列排名的column
monetary_df['monetary_rank'] = monetary_df['profit'].rank(method='first', ascending=False)
monetary_df

##RFM Score Rank

In [ ]:
# 提取每個的排名columns

RFM_df = recency_df[['customer_id', 'recency_rank']].merge(
    frequency_df[['customer_id', 'frequency_rank']], on='customer_id').merge(
    monetary_df[['customer_id', 'monetary_rank']], on='customer_id')
RFM_df

In [ ]:
RFM_df['RFM_score'] = (RFM_df['recency_rank'] + RFM_df['frequency_rank'] + RFM_df['monetary_rank'])/3
RFM_df

In [ ]:
# 定義條件

conditions = [
    (RFM_df['RFM_score'] >= 1) & (RFM_df['RFM_score'] <= 1000),
    (RFM_df['RFM_score'] > 1000) & (RFM_df['RFM_score'] <= 3000),
    (RFM_df['RFM_score'] > 3000) & (RFM_df['RFM_score'] <= 4000)
]

# 對應的分類
choices = ['High Value', 'Medium Value', 'Low Value']

# 新增一欄 customer_segment
RFM_df['RFM_label'] = np.select(conditions, choices, default='Unknown')
RFM_df

# RFM Model EDA

### 提取 Data

In [ ]:
join_RFM_df = pd.merge(RFM_df, join_df, on='customer_id', how = 'inner')
join_RFM_df.head()

In [ ]:
join_RFM_df.info()

### 單一維度

In [ ]:
# Dist Plot

sns.distplot(join_RFM_df['RFM_score'], bins=30, rug=True)
plt.show()

In [ ]:
# Count Plot

sns.countplot(data=join_RFM_df, x='RFM_label')
plt.show()

### 兩個維度：x軸cate, y軸num

*   Strip Plot
*   Box Plot
*   Count Plot (x & hue都是cate)

In [ ]:
# Box Plot
sns.boxplot(data=join_RFM_df, x='RFM_label', y='profit')
plt.show()

In [ ]:
# Count Plot
sns.countplot(data=join_RFM_df, x='RFM_label', hue='gender')
plt.show()

### 三個維度： ：x軸cate, y軸num, hue
*   Strip Plot 散點圖
*   Box Plot 箱型圖(聚合型)
*   Bar Plot 條形圖

In [ ]:
# Bar Plot
sns.barplot(data=join_RFM_df, x='RFM_label', y='profit', hue='gender')
plt.show()

### 三個維度：x軸 ＆ y軸 num, hue

*   Joint Plot
*   Point Plot

### 多個維度： Cat Plot

In [ ]:
sns.set(style='whitegrid', palette='pastel')
plt.figure(figsize=(20, 10))


plot_3 = sns.catplot(x='brand', y='profit', data=join_RFM_df, hue='gender', col='RFM_label', row='state',kind='bar')
plot_3.set(xlabel="brand",ylabel="profit")
for ax in plot_3.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

### 多個Num V: Heat Plot

In [ ]:
sns.heatmap(join_RFM_df[["recency_rank","frequency_rank","monetary_rank"]].corr(), annot=True)
plt.show()

# RFM Segmentation Analysis

### 提取Data

In [ ]:
df_rfm = join_RFM_df.groupby(['customer_id'])['profit'].sum().reset_index()
df_rfm['rank_M'] = join_RFM_df['profit'].rank(ascending = False)

In [ ]:
df_52 = join_RFM_df[(join_RFM_df['CohortMonth'] == '2024-07-01') & (join_RFM_df['CohortIndex'] == 2)]
df_28 = join_RFM_df[(join_RFM_df['CohortMonth'] == '2024-07-01') & (join_RFM_df['CohortIndex'] == 3)]

### 單一維度：y = 'profit’

In [ ]:
# Box Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))

sns.boxplot(data=df_52, y='RFM_score', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("RFM_score")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.boxplot(data=df_28, y='RFM_score', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

In [ ]:
# Count Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 6))

sns.countplot(data=df_52, x='RFM_label', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Count")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.countplot(data=df_28, x='RFM_label', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

### 兩個維度：x軸cate, y軸num
- Strip Plot
- Box Plot
- Count Plot (x & hue都是Cate)

In [ ]:
# Box Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.boxplot(data=df_52, x='brand', y='RFM_score', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("RFM_score")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.boxplot(data=df_28, x='brand', y='RFM_score', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()


In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.boxplot(data=df_52, x='RFM_label', y='profit', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.boxplot(data=df_28, x='RFM_label', y='profit', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

In [ ]:
# Count Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.countplot(data=df_52, x='RFM_label', hue='gender', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Count")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.countplot(data=df_28, x='RFM_label', hue='gender', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

### 三個維度：x軸cate, y軸num, hue
- Strip Plot 散點圖
- Box Plot 箱型圖(聚合型)
- Bar Plot 條形圖

In [ ]:
# Bar Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.barplot(data=df_52, x='RFM_label', y='RFM_score', hue='gender', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("RFM_score")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.barplot(data=df_28, x='RFM_label', y='RFM_score', hue='gender', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

In [ ]:
# Box Plot
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10), sharey=True)

sns.boxplot(data=df_52, x='state', y='profit', hue='RFM_label', ax=axes[0], color='skyblue')
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_ylabel("Profit")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.boxplot(data=df_28, x='state', y='profit', hue='RFM_label', ax=axes[1], color='lightcoral')
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_ylabel("")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()

### 三個維度： x軸 & y軸 Num, hue
- Joint Plot
- Point Plot

In [ ]:
# 畫這兩個圖的意義不大

### 多個維度：Cat Plot

In [ ]:
sns.set(style='whitegrid', palette='pastel')
plt.figure(figsize=(20, 10))


plot_52 = sns.catplot(x='brand', y='profit', data=df_52, hue='gender', col='RFM_label', row='state',kind='bar')
plot_52.set(xlabel="brand",ylabel='profit')
for ax in plot_52.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

plot_28 = sns.catplot(x='brand', y='profit', data=df_28, hue='gender', col='RFM_label', row='state',kind='bar')
plot_28.set(xlabel="brand",ylabel='profit')
for ax in plot_28.axes.flatten():
    ax.tick_params(axis='x', rotation=45)

plt.show()

### 多個Num V維度：Heat Plot

In [ ]:
df_52[["recency_rank","frequency_rank","monetary_rank"]].corr()
df_28[["recency_rank","frequency_rank","monetary_rank"]].corr()

In [ ]:
sns.set(style="whitegrid")
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(10, 4), sharey=True)

sns.heatmap(df_52[["recency_rank","frequency_rank","monetary_rank"]].corr(), annot=True, ax=axes[0])
axes[0].set_title("Cohort Index 2 (52.5%) - September")
axes[0].set_xlabel("2024-07 Cohort Group")

sns.heatmap(df_28[["recency_rank","frequency_rank","monetary_rank"]].corr(), annot=True, ax=axes[1])
axes[1].set_title("Cohort Index 3 (28.8%) - October")
axes[1].set_xlabel("2024-07 Cohort Group")

plt.tight_layout()
plt.show()